In [ ]:
import pandas as pd
import joblib
import math
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import xgboost as xgb

In [ ]:
# function to calculate absolute humidity
def ah1(t, rh):
    ah = (1322.9*(rh/100)*math.e**(t/(t+238.3)*17.2694) / (t+273.15))
    return ah

In [ ]:
# function to read sheets from ScienceDirect into Pandas dataframe
def rh_reader(fn, sheet):
    df = pd.read_excel(fn, sheet_name=sheet,
                          names=["date", "temp_in", "rh_in"], header=None,
                          parse_dates=True, index_col=0, skiprows=2, usecols=[0,1,2])
    return df

In [ ]:
# load data from the three storage halls. Sheets come from ScienceDirect
df_rh_1 = rh_reader('NM_S_OP_CI_CO_EN_2009-01_2020-12_002.xlsx', 'Ørholm-magasin-O-P-1-1-b')
df_rh_2 = rh_reader('NM_S_OP2_CI_CO_EN_2009-01_2020-12_010.xlsx', 'Ørholm-magasin-O-P-1-2-b')
df_rh_3 = rh_reader('NM_S_OP3_CI_2009-01_2021-05_016.xlsx', 'Ørholm-magasin-O-P-1-3-b')

In [ ]:
# rename columns to avoid name clashes when we merge the dataframes
df_rh_1.rename(columns={'temp_in': 'temp_1', 'rh_in': 'rh_1'}, inplace = True)
df_rh_2.rename(columns={'temp_in': 'temp_2', 'rh_in': 'rh_2'}, inplace = True)
df_rh_3.rename(columns={'temp_in': 'temp_3', 'rh_in': 'rh_3'}, inplace = True)

# merge into one dataframe
df = df_rh_1.join([df_rh_2, df_rh_3], how='inner')

In [ ]:
# we have three measurements that we assume should be somewhat similar.
# To be roboust against "drifting" thermometers or hygrometers, we look at the median/middle one of the three.
# we also calculate the "middle absolute humidity" for each.
df['rh_median'] = df[['rh_1', 'rh_2', 'rh_3']].median(axis=1)
df['temp_median'] = df[['temp_1', 'temp_2', 'temp_3']].median(axis=1)
df['ah_median'] = df.apply(lambda x: ah1(x.temp_median, x.rh_median), axis = 1)

In [ ]:
## Define auxillary values that are potentially interesting for time series analysis

# define "rolling mean" of humidity over 24 hour periods
df['rh_median_24_mean'] = df.rh_median.rolling(24, min_periods=1).mean()
# define exponential weighted mean for humidity over 24 hr perionds
df['rh_median_24_ewm_mean'] = df.rh_median.ewm(span=24).mean()
# define medians of humidity over 24 hr periods
df['rh_median_24_median'] = df.rh_median.rolling(24, min_periods=1).median()
# Minimum and maximum values of humidity medians over 24 hour periods.
df['rh_median_24_min'] = df.rh_median.rolling(24, min_periods=1).min()
df['rh_median_24_max'] = df.rh_median.rolling(24, min_periods=1).max() 

In [ ]:
# define prediction target: take mean of rolling 24-hr windows, and shift back a day.
df['rh_median_mean_rolling'] = df.rh_median.rolling(window=24).mean().shift(-23)
# if this gets too high or low, we will want to raise a flag, so y_low and y_high 
# will be the labels of our classifier.
df['y_low'] = df['rh_median_mean_rolling'] < 40
df['y_high'] = df['rh_median_mean_rolling'] > 60

In [ ]:
# now that labels are defined, we drop that column again
df = df.drop(['rh_median_mean_rolling'], axis=1) 
# also drop original data; it is encoded in the aux columns
df = df.drop(['temp_1', 'temp_2', 'temp_3', 'rh_1', 'rh_2', 'rh_3'], axis=1)

In [ ]:
# xgboost takes a numpy feature array as input. That will be the same for the 'high' and 'low' models
features = df.copy()
features = features.drop(['y_high', 'y_low'], axis=1)
X = np.array(features)

In [ ]:
# for both the 'high' and 'low' model, we do a grid search to find the best hyperparameters for 
# xgBoost classifier, and dump it to a file that we can subsequently use for predictions
for s in ['y_high', 'y_low']:
    # pick the right label
    y = df[s].to_numpy()
    # split data into train and test sets to build model.
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify=y)
    param_fixed = {'objective':'binary:logistic', 'n_jobs':20}
    clf = xgb.XGBClassifier(**param_fixed)
    # these are the hyperparameters we do grid search on:
    parameters = {
        'max_depth' : range(2,10,1),
        'n_estimators' : range(60,220,40),
        'learning_rate': [0.1, 0.01, 0.05]
    }
    # this will take several hours, even on quite powerful machines. 
    # Consider running in a standalone program that you can leave running overnight.
    grid = GridSearchCV(
        estimator=clf,
        param_grid=parameters,
        scoring = 'roc_auc',
        n_jobs = 10,
        cv = 10,
        verbose=True
    ).fit(train_x, train_y)
    # choose the best estimator from the grid search and predict on the test set:
    best_pipe = grid.best_estimator_
    predictions = best_pipe.predict(test_x)
    # print the best hyperparams and evaluate model:
    print(f"best params: {grid.best_params_}")
    print(classification_report(test_y, predictions))
    # dump the best model to model file that can subsequently be used to make predictions
    joblib.dump(clf, f"orholm-data-xgb-model-{s}.joblib")

## Output (run from a standalone program)

### y_high:

```
Fitting 5 folds for each of 48 candidates, totalling 240 fits
best params: {'learning_rate': 0.05, 'max_depth': 100, 'n_estimators': 500}
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     22728
           1       0.92      0.92      0.92      3297

    accuracy                           0.98     26025
   macro avg       0.95      0.95      0.95     26025
weighted avg       0.98      0.98      0.98     26025
```

### y_low:

```
Fitting 5 folds for each of 48 candidates, totalling 240 fits
best params: {'learning_rate': 0.1, 'max_depth': 50, 'n_estimators': 500}
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     24664
           1       0.94      0.93      0.93      1361

    accuracy                           0.99     26025
   macro avg       0.97      0.96      0.96     26025
weighted avg       0.99      0.99      0.99     26025
```